# TALLER Clase 12

Alejandro Mantilla Redondo - 201711304

Juan Betancourt - 201632544


Usando los datos Carseats: Se usan como variables explicativas continuas Advertising e Income.
Como variables categóricas se usan Urban y ShelveLoc.

Se considera un modelo con todas las dummies y todas las interacciones.

In [1]:
datos <- ISLR::Carseats

# Excluimos la columna shelve_loc_good para no
# introducir multicolinealidad al modelo.
shelve_loc_bad <- as.double(datos$ShelveLoc == "Bad")
shelve_loc_medium <- as.double(datos$ShelveLoc == "Medium")

urban <- as.double(datos$Urban == "Yes")

datos <- cbind(datos[c(1, 4, 3)], urban, shelve_loc_bad, shelve_loc_medium)

colnames(datos) <- c("Sales",
                     "Advertising",
                     "Income",
                     "Urban",
                     "Shelve_loc_bad",
                     "Shelve_loc_medium")

datos <- cbind(datos,
               datos["Shelve_loc_bad"]    * datos["Advertising"],
               datos["Shelve_loc_medium"] * datos["Advertising"],
               datos["Urban"]             * datos["Advertising"],
               datos["Shelve_loc_bad"]    * datos["Income"],
               datos["Shelve_loc_medium"] * datos["Income"],
               datos["Urban"]             * datos["Income"])

colnames(datos) <- c("Sales", 
                     "Advertising",
                     "Income",
                     "Urban",
                     "Shelve_loc_bad",
                     "Shelve_loc_medium",
                     "Advertising:Shelve_loc_bad",
                     "Advertising:Shelve_loc_medium",
                     "Advertising:Urban",
                     "Income:Shelve_loc_bad",
                     "Income:Shelve_loc_medium",
                     "Income:Urban")

head(datos)

,Sales,Advertising,Income,Urban,Shelve_loc_bad,Shelve_loc_medium,Advertising:Shelve_loc_bad,Advertising:Shelve_loc_medium,Advertising:Urban,Income:Shelve_loc_bad,Income:Shelve_loc_medium,Income:Urban
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,9.50,11,73,1,1,0,11,0,11,73,0,73
2,11.22,16,48,1,0,0,0,0,16,0,0,48
3,10.06,10,35,1,0,1,0,10,10,0,35,35
4,7.40,4,100,1,0,1,0,4,4,0,100,100
5,4.15,3,64,1,1,0,3,0,3,64,0,64
6,10.81,13,113,0,1,0,13,0,0,113,0,0


Tomamos como modelo completo el descrito por la siguiente expresión. Excluimos las interacciones entre variables de mismo tipo (continuas con continuas, binarias con binarias).

\begin{align*}
  \text{Sales}\ =\ & \beta_{0}\cdot x_{0} + \beta_{\text{Advertising}}\cdot x_{\text{Advertising}} + \beta_{\text{Income}}\cdot x_{\text{Income}} +\\
               & \gamma_{\text{Shelve_loc_bad}}\cdot z_{\text{Shelve_loc_bad}} + \gamma_{\text{Shelve_loc_medium}}\cdot z_{\text{Shelve_loc_medium}} + \gamma_{\text{Urban}}\cdot z_{\text{Urban}} + \\
               & \eta_{\text{Urban}\cdot \text{Advertising}}\cdot \left(z_{\text{Urban}}\cdot x_{\text{Advertising}}\right) + \eta_{\text{Urban}\cdot \text{Income}}\cdot \left(z_{\text{Urban}}\cdot x_{\text{Income}}\right) +\\
               & \eta_{\text{Shelve_loc_bad}\cdot \text{Advertising}}\cdot \left(z_{\text{Shelve_loc_bad}}\cdot x_{\text{Advertising}}\right) + \eta_{\text{Shelve_loc_bad}\cdot \text{Income}}\cdot \left(z_{\text{Shelve_loc_bad}}\cdot x_{\text{Income}}\right) +\\
               & \eta_{\text{Shelve_loc_medium}\cdot \text{Advertising}}\cdot \left(z_{\text{Shelve_loc_medium}}\cdot x_{\text{Advertising}}\right) + \eta_{\text{Shelve_loc_medium}\cdot \text{Income}}\cdot \left(z_{\text{Shelve_loc_medium}}\cdot x_{\text{Income}}\right).
\end{align*}

In [2]:
modelo <- lm(Sales ~ (Advertising + Income) *
             (Shelve_loc_bad + Shelve_loc_medium + Urban), data = datos)

summary(modelo)


Call:
lm(formula = Sales ~ (Advertising + Income) * (Shelve_loc_bad + 
    Shelve_loc_medium + Urban), data = datos)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.1366 -1.5673  0.0162  1.5435  6.2686 

Coefficients:
                               Estimate Std. Error t value Pr(>|t|)    
(Intercept)                    8.529838   0.817714  10.431  < 2e-16 ***
Advertising                    0.124337   0.044591   2.788  0.00556 ** 
Income                         0.011296   0.010397   1.086  0.27796    
Shelve_loc_bad                -5.184761   0.946788  -5.476 7.83e-08 ***
Shelve_loc_medium             -3.654387   0.789649  -4.628 5.04e-06 ***
Urban                          0.658878   0.651054   1.012  0.31216    
Advertising:Shelve_loc_bad    -0.030251   0.050147  -0.603  0.54670    
Advertising:Shelve_loc_medium -0.003157   0.042007  -0.075  0.94014    
Advertising:Urban             -0.027971   0.037536  -0.745  0.45662    
Income:Shelve_loc_bad          0.010740   0.012051   

Definimos una función que ejecute la prueba $F$ generalizada.

In [3]:
prueba_f_gen <- function(y, X, A, c){

    n <- length(X[, 1])
    p <- length(X[1,])
    q <- length(A[, 1])

    xtx <- t(X) %*% X
    H <- solve(xtx)

    beta_hat <- H %*% t(X) %*% y

    y_hat <- X %*% beta_hat

    s2 <- sum((y - y_hat)^2) / (n - p)

    beta_r <- beta_hat +
              (H %*% t(A) %*% solve(A %*% H %*% t(A)) %*% (c -
                                                           (A %*% beta_hat)))

    ep <- (sum((y_hat - (X %*% beta_r)) ^ 2) / q) / s2

    pvalue <- 1 - pf(ep, q, (n - p))

    return(c(ep, pvalue))
}

Definimos nuestras matrices $X$ y $Y$.

In [4]:
X <- as.matrix(cbind(1, datos[, -1]))
head(X)
y <- as.matrix(datos["Sales"])
head(y)

,1,Advertising,Income,Urban,Shelve_loc_bad,Shelve_loc_medium,Advertising:Shelve_loc_bad,Advertising:Shelve_loc_medium,Advertising:Urban,Income:Shelve_loc_bad,Income:Shelve_loc_medium,Income:Urban
1,1,11,73,1,1,0,11,0,11,73,0,73
2,1,16,48,1,0,0,0,0,16,0,0,48
3,1,10,35,1,0,1,0,10,10,0,35,35
4,1,4,100,1,0,1,0,4,4,0,100,100
5,1,3,64,1,1,0,3,0,3,64,0,64
6,1,13,113,0,1,0,13,0,0,113,0,0


,Sales
1,9.50
2,11.22
3,10.06
4,7.40
5,4.15
6,10.81


## 1. Es cierto que si el sitio es urbano o la posición en el estante no influyen sobre las ventas?

Consideramos las siguientes hipótesis.

\begin{align*}
    H_{1}:\ & \gamma_{\text{Urban}} + \gamma_{\text{Shelve_loc_bad}} + \gamma_{\text{Shelve_loc_medium}} = 0 \ \land \\
          & \eta_{\text{Urban} \cdot \text{Advertising}} + \eta_{\text{Shelve_loc_medium}\cdot \text{Advertising}} + \eta_{\text{Shelve_loc_bad}\cdot \text{Advertising}} = 0  \ \land \\
          & \eta_{\text{Urban}\cdot \text{Income}} + \eta_{\text{Shelve_loc_medium}\cdot \text{Income}} + \eta_{\text{Shelve_loc_bad}\cdot \text{Income}} = 0, \\
    H_{0}:\ & \neg H_{1}.
\end{align*}

In [5]:
A <- matrix(c(0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1),
            nrow = 3, byrow = TRUE)

c <- matrix(0, nrow = 3, ncol = 1)

resultado <- prueba_f_gen(y, X, A, c)

print(paste("El estadístico de prueba es:", resultado[1], sep = " "))
print(paste("El p-value respectivo es: ", resultado[2], sep = " "))

[1] "El estadístico de prueba es: 52.723648858676"
[1] "El p-value respectivo es:  0"


Al evaluar la prueba de hipótesis a través del estadístico de prueba es posible ver la probabilidad de que una variable aleatoria con distribución $F_{3,388}$ sea mayor al estadístico de prueba es $0 < \alpha=0.05$. Esto quiere decir que hay suficiente evidencia estadística para rechazar la hipótesis nula que se planteó. En términos prácticos del problema, podemos concluir que no es cierto que si el sitio es urbano o la posición en el estante no influyen sobre las ventas.

## 2. Es cierto que la posición en el estante no influye sobre las ventas?

Consideramos las siguientes hipótesis.

\begin{align*}
    H_{1}:\ & \gamma_{\text{Shelve_loc_bad}} = 0 \ \land \\
            & \gamma_{\text{Shelve_loc_medium}} = 0 \ \land \\
            & \eta_{\text{Shelve_loc_medium}\cdot \text{Advertising}} = 0 \ \land \\ 
            & \eta_{\text{Shelve_loc_bad}\cdot \text{Advertising}} = 0 \ \land \\
            & \eta_{\text{Shelve_loc_medium}\cdot \text{Income}} = 0 \ \land \\
            & \eta_{\text{Shelve_loc_bad}\cdot \text{Income}} = 0, \\
    H_{0}:\ & \neg H_{1}.
\end{align*}

In [6]:
A <- matrix(c(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
            nrow = 6, byrow = TRUE)

c <- matrix(0, nrow = 6, ncol = 1)

resultado <- prueba_f_gen(y, X, A, c)

paste("El estadístico de prueba es:", resultado[1], sep = " ")
paste("El p-value respectivo es: ", resultado[2], sep = " ")

[1] "El estadístico de prueba es: 33.399225143868"

[1] "El p-value respectivo es:  0"

Al evaluar la prueba de hipótesis a través del estadístico de prueba es posible ver la probabilidad de que una variable aleatoria con distribución $F_{6,388}$ sea mayor al estadístico de prueba es $0 < \alpha=0.05$. Esto quiere decir que hay suficiente evidencia estadística para rechazar la hipótesis nula que se planteó. En términos prácticos del problema, podemos concluir que no es cierto que la posición en el estante no influya sobre las ventas.

## 3. Es cierto que los modelos en todas las celdas son paralelos (los efectos no cambian)?

Consideramos las siguientes hipótesis.

\begin{align*}
    H_{1}:\ & \eta_{\text{Urban}\cdot \text{Advertising}} = 0 \ \land \\
            & \eta_{\text{Shelve_loc_bad}\cdot \text{Advertising}} = 0 \ \land \\
            & \eta_{\text{Shelve_loc_medium}\cdot \text{Advertising}} = 0 \ \land \\
            & \eta_{\text{Urban}\cdot \text{Income}} = 0 \ \land \\
            & \eta_{\text{Shelve_loc_bad}\cdot \text{Income}} = 0 \ \land \\
            & \eta_{\text{Shelve_loc_medium}\cdot \text{Income}} = 0, \\
    H_{0}:\ & \neg H_{1}.
\end{align*}

In [7]:
A <- matrix(c(0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1),
            nrow = 6, byrow = TRUE)

c <- matrix(0, nrow = 6, ncol = 1)

resultado <- prueba_f_gen(y, X, A, c)

paste("El estadístico de prueba es:", resultado[1], sep = " ")
paste("El p-value respectivo es: ", resultado[2], sep = " ")

[1] "El estadístico de prueba es: 0.605767462892683"

[1] "El p-value respectivo es:  0.725757645321409"

Al evaluar la prueba de hipótesis a través del estadístico de prueba es posible ver la probabilidad de que una variable aleatoria con distribución $F_{6,388}$ sea mayor al estadístico de prueba es $0.73 > \alpha=0.05$. Esto quiere decir que hay suficiente evidencia estadística para no rechazar la hipótesis nula que se planteó. En términos prácticos del problema, podemos concluir que es cierto que los modelos en todas las celdas son paralelos.

## 4. Son los modelos para las ventas en una zona urbana con posición media en el estante, y en una zona rural con posición buena iguales?

Consideramos las siguientes hipótesis.

\begin{align*}
    H_{1}:\ & \gamma_{\text{Urban}} + \gamma_{\text{Shelve_loc_medium}} = 0 \ \land \\
            & \eta_{\text{Urban}\cdot \text{Advertising}} + \eta_{\text{Shelve_loc_medium}\cdot \text{Advertising}} = 0 \ \land\\
            & \eta_{\text{Urban}\cdot \text{Income}} + \eta_{\text{Shelve_loc_medium}\cdot \text{Income}} = 0, \\
    H_{0}:\ & \neg H_{1}.
\end{align*}

In [8]:
A <- matrix(c(0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1),
            nrow = 3, byrow = TRUE)

c <- matrix(0, nrow = 3, ncol = 1)

resultado <- prueba_f_gen(y, X, A, c)

paste("El estadístico de prueba es:", resultado[1], sep = " ")
paste("El p-value respectivo es: ", resultado[2], sep = " ")

[1] "El estadístico de prueba es: 17.2384726794462"

[1] "El p-value respectivo es:  1.57975965642265e-10"

Al evaluar la prueba de hipótesis a través del estadístico de prueba es posible ver la probabilidad de que una variable aleatoria con distribución $F_{3,388}$ sea mayor al estadístico de prueba es $1.5*10^{-10} < \alpha=0.05$. Esto quiere decir que hay suficiente evidencia estadística para rechazar la hipótesis nula que se planteó. En términos prácticos del problema, podemos concluir que no es cierto que los modelos para las ventas en una zona urbana con posición media en el estante, y en una zona rural con posición buena son iguales.

## 5. Es cierto que el efecto de la publicidad es igual en la zona rural que la urbana?

Consideramos las siguientes hipótesis.

\begin{align*}
    H_{1}:\ & \eta_{\text{Urban} \cdot \text{Advertising}} = 0, \\
    H_{0}:\ & \neg H_{1}.
\end{align*}

In [9]:
A <- matrix(c(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0),
            nrow = 1, byrow = TRUE)

c <- matrix(0, nrow = 1, ncol = 1)

resultado <- prueba_f_gen(y, X, A, c)

paste("El estadístico de prueba es:", resultado[1], sep = " ")
paste("El p-value respectivo es: ", resultado[2], sep = " ")

[1] "El estadístico de prueba es: 0.555273776963337"

[1] "El p-value respectivo es:  0.456621940822689"

Al evaluar la prueba de hipótesis a través del estadístico de prueba es posible ver la probabilidad de que una variable aleatoria con distribución $F_{1,388}$ sea mayor al estadístico de prueba es $0.45 > \alpha=0.05$. Esto quiere decir que hay suficiente evidencia estadística para no rechazar la hipótesis nula que se planteó. En términos prácticos del problema, podemos concluir que es cierto que el efecto de la publicidad es igual en la zona rural que la urbana.

## 6. Es cierto que el efecto de la publicidad es igual en la zona rural con posición media que en la zona urbana con posición mala?

Consideramos las siguientes hipótesis.

\begin{align*}
    H_{1}:\ & \eta_{\text{Urban} \cdot \text{Advertising}} + \eta_{\text{Shelve_loc_bad}\cdot \text{Advertising}} = \eta_{\text{Shelve_loc_medium}\cdot \text{Advertising}}, \\
    H_{0}:\ & \neg H_{1}.
\end{align*}

In [10]:
A <- matrix(c(0, 0, 0, 0, 0, 0, 1, 1, -1, 0, 0, 0),
            nrow = 1, byrow = TRUE)

c <- matrix(0, nrow = 1, ncol = 1)

resultado <- prueba_f_gen(y, X, A, c)

paste("El estadístico de prueba es:", resultado[1], sep = " ")
paste("El p-value respectivo es: ", resultado[2], sep = " ")

[1] "El estadístico de prueba es: 0.00353881901256342"

[1] "El p-value respectivo es:  0.952594070991012"

Al evaluar la prueba de hipótesis a través del estadístico de prueba es posible ver la probabilidad de que una variable aleatoria con distribución $F_{1,388}$ sea mayor al estadístico de prueba es $0.95 > \alpha=0.05$. Esto quiere decir que hay suficiente evidencia estadística para no rechazar la hipótesis nula que se planteó. En términos prácticos del problema, podemos concluir que es cierto que el efecto de la publicidad es igual en la zona rural con posición media que en la zona urbana con posición mala.